In [1]:
#note that the adjacency matrix for a graph is a 0/1 matrix
#0/1 exists in any field, so we can view A \in (F_p)^{n^2} \cong F_{p^{n^2}} = F_q
#thus every adjacency matrix A can be viewed as an element of F_q

In [51]:
from sage.misc.flatten import flatten
def graph_embedding(A,p):
    assert p in Primes()
    n = len(A)
    q = p**(n**2)
    alpha = GF(q).gen()
    flat_A = flatten(A)
    return sum([flat_A[k]*alpha**k for k in range(n^2)])

In [52]:
graph_embedding([[0,1],[1,0]],5)

z4^2 + z4